In [3]:
import logging
import threading
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import bif
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.db import Database
from iotfunctions.enginelog import EngineLogging
from iotfunctions import estimator
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from iotfunctions.enginelog import EngineLogging
from iotfunctions import pipeline as pp
from iotfunctions.pipeline import Db2DataWriter, JobController, DataWriterFile, DataAggregator

import datetime as dt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline

/home/markus/.local/lib/python3.7/site-packages/iotfunctions/bif.py:1899: UserWarning: IoTCalcSettings is deprecated. Use entity type constants instead of a metadata provider to set entity type properties
  warnings.warn(('IoTCalcSettings is deprecated. Use entity type constants'


In [4]:
credentials = {
  "tenantId": "AnalyticsServiceDev",
  "as_api_host": "https://api-dev.connectedproducts.internetofthings.ibmcloud.com",
  "as_api_key": "a-69xgm4-8bdgtvnsv4",
  "as_api_token": "9X_tMKdupOiJ!mzaPV",
  "config" : {
      "objectStorageEndpoint" : "https://s3-api.us-geo.objectstorage.softlayer.net",
      "bos_runtime_bucket" : "analytics-runtime-analyticsservicedev-799d2008b460",
      "bos_logs_bucket" : "analytics-logs-analyticsservicedev-32703c52ec8b"
  },
  "objectStorage": {
      "username" : "58ddd86b5de8468b819d385046f17033",
      "password" : "ee0d6c5521ce9ff100f91b0e37d4eb8cc1a038b5a6d05b38",
      "region" : "us",
      "endpoint" : "https://s3-api.us-geo.objectstorage.softlayer.net"
  },
  "db2": {
    "username": "bluadmin",
    "password": "ZmM5MmE5NmZkZGZl",
    "databaseName": "BLUDB",
    "port": 50000,
    "httpsUrl": "https://dashdb-enterprise-yp-dal13-74.services.dal.bluemix.net:50000",
    "host": "dashdb-enterprise-yp-dal13-74.services.dal.bluemix.net"
  },
  "postgres": None
}
EngineLogging.configure_console_logging(logging.DEBUG)

2019-09-27T14:01:02.760 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10


In [5]:
db_schema = None
db = Database(credentials=credentials)
print (db)
# extend iotfunctions/db.py to support postgres connection string in credentials

2019-09-27T14:01:04.044 DEBUG iotfunctions.db.__init__ Unable to locate message_hub credentials. Database object created, but it will not be able interact with message hub.
2019-09-27T14:01:04.045 DEBUG iotfunctions.db.__init__ created a CosClient object
2019-09-27T14:01:06.623 DEBUG iotfunctions.db.__init__ Db connection established
2019-09-27T14:01:06.625 DEBUG urllib3.connectionpool._new_conn Starting new HTTPS connection (1): api-dev.connectedproducts.internetofthings.ibmcloud.com


/home/markus/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-09-27T14:01:10.955 DEBUG urllib3.connectionpool._make_request https://api-dev.connectedproducts.internetofthings.ibmcloud.com:443 "GET /api/meta/v1/AnalyticsServiceDev/entityType HTTP/1.1" 200 None
2019-09-27T14:01:11.510 DEBUG iotfunctions.db.http_request http request successful. status 200


In [4]:
table = db.get_table("MARKUS_TEST_ROBOT1")
start_ts = dt.datetime.utcnow() - dt.timedelta(days=1)
end_ts = dt.datetime.utcnow()
df = db.read_table(table, None, None, None, "evt_timestamp", start_ts, end_ts)
print (table)

/home/markus/.local/lib/python3.7/site-packages/sqlalchemy/engine/reflection.py:913: SAWarning: index key 'sqlnotapplicable' was not located in columns for table 'MARKUS_TEST_ROBOT1'
  "columns for table '%s'" % (flavor, c, table_name)


MARKUS_TEST_ROBOT1


In [5]:
print (df.head(10))

Empty DataFrame
Columns: [deviceid, evt_timestamp, devicetype, logicalinterface_id, eventtype, format, updated_utc, plant_code, torque, tool_type, load, speed, travel_time, acc]
Index: []


In [6]:
# Generate 5 mins of data in table 'testdata' with a single additional column of TestData
EngineLogging.configure_console_logging(logging.DEBUG)
jobsettings = {}
#jobsettings = {'_timestamp' : 'TIMESTAMP'}
et = metadata.EntityType('markus_testdata', db, 
                         bif.EntityDataGenerator(output_item='my_test_gen'),
                         
                         Column('TestData',Float()),
                         **jobsettings)

#start_date = dt.datetime.utcnow() - dt.timedelta(days=1)
#et.exec_local_pipeline(start_ts = start_date)
df = et.generate_data()

2019-09-26T18:52:59.912 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10
2019-09-26T18:52:59.914 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 2.0.3
2019-09-26T18:52:59.917 DEBUG iotfunctions.pipeline.__init__ Starting trace
2019-09-26T18:52:59.919 DEBUG iotfunctions.pipeline.__init__ Trace name: auto_trace_markus_testdata_20190926165259
2019-09-26T18:52:59.920 DEBUG iotfunctions.pipeline.__init__ auto_save None
2019-09-26T18:52:59.923 WARNING iotfunctions.metadata.__init__ No _db_schema specified in **kwargs. Usingdefault database schema.
2019-09-26T18:52:59.924 DEBUG iotfunctions.util.categorize_args categorizing arguments


/home/markus/.local/lib/python3.7/site-packages/sqlalchemy/engine/reflection.py:913: SAWarning: index key 'sqlnotapplicable' was not located in columns for table 'markus_testdata'
  "columns for table '%s'" % (flavor, c, table_name)


2019-09-26T18:53:02.692 DEBUG iotfunctions.metadata.__init__ Initialized entity type 
EntityType:markus_testdata
Functions:
EntityDataGenerator at granularity None required inputs not evaluated yet outputs produced not evaluated yet on schedule None
Granularities:
No schedules metadata
2019-09-26T18:53:02.694 DEBUG iotfunctions.metadata.generate_data Generating data for markus_testdata with metrics ['TestData'] and dimensions ['eventtype'] and dates []
2019-09-26T18:53:02.715 DEBUG iotfunctions.automation.get_data Generated 6 rows of time series data from 2019-09-26 16:48:02.695417 to 2019-09-26 16:53:02.695417
2019-09-26T18:53:03.819 INFO iotfunctions.db.write_frame Wrote data to table markus_testdata 


In [4]:
#jobsettings = {'_timestamp' : 'TIMESTAMP'}
jobsettings = {}
et2 = metadata.EntityType('markus_testdata', db, 
                          Column('TestData',Float()),
                          **jobsettings)
et2.get_data()
print (et2)

2019-09-27T12:07:03.800 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 2.0.3
2019-09-27T12:07:03.801 DEBUG iotfunctions.pipeline.__init__ Starting trace
2019-09-27T12:07:03.801 DEBUG iotfunctions.pipeline.__init__ Trace name: auto_trace_markus_testdata_20190927100703
2019-09-27T12:07:03.802 DEBUG iotfunctions.pipeline.__init__ auto_save None
2019-09-27T12:07:03.803 WARNING iotfunctions.metadata.__init__ No _db_schema specified in **kwargs. Usingdefault database schema.
2019-09-27T12:07:03.803 DEBUG iotfunctions.util.categorize_args categorizing arguments


/home/markus/.local/lib/python3.7/site-packages/sqlalchemy/engine/reflection.py:913: SAWarning: index key 'sqlnotapplicable' was not located in columns for table 'markus_testdata'
  "columns for table '%s'" % (flavor, c, table_name)


2019-09-27T12:07:07.025 DEBUG iotfunctions.metadata.__init__ Initialized entity type 
EntityType:markus_testdata
Functions:
Granularities:
No schedules metadata
2019-09-27T12:07:07.521 DEBUG iotfunctions.metadata.index_df Indexed dataframe on id, evt_timestamp

EntityType:markus_testdata
Functions:
Granularities:
No schedules metadata


In [5]:
from iotfunctions import pipeline as pp
#job = pp.JobController(et)
#job.execute()

In [6]:
print (df)

NameError: name 'df' is not defined

In [7]:
# read it back
table = db.get_table("markus_testdata")
start_ts = dt.datetime.utcnow() - dt.timedelta(days=1)
end_ts = dt.datetime.utcnow()
df_in = db.read_table(table, None, None, None, "evt_timestamp", start_ts, end_ts)
print (table)
print (df_in.shape)

markus_testdata
(12, 8)


In [8]:
print (df_in.tail(5))

   deviceid              evt_timestamp       devicetype logicalinterface_id  \
7     73004 2019-09-26 16:49:02.695417  markus_testdata                       
8     73004 2019-09-26 16:50:02.695417  markus_testdata                       
9     73002 2019-09-26 16:51:02.695417  markus_testdata                       
10    73001 2019-09-26 16:52:02.695417  markus_testdata                       
11    73000 2019-09-26 16:53:02.695417  markus_testdata                       

   eventtype format updated_utc  TestData  
7         te               None  0.372196  
8         ne               None -0.391822  
9         te               None -1.279792  
10        np               None  1.143055  
11        et               None -0.361705  


In [9]:
# check whether the data for the last 5 minutes is the same - must return True
print (np.array_equal(df['TestData'].tail(5), df_in['TestData'].tail(5)))

NameError: name 'df' is not defined

In [20]:
print (et._functions)
#del (et2)

NameError: name 'et' is not defined

In [6]:

jobsettings = {}
et2 = metadata.EntityType('markus_testdata', db, 
                          Column('TestData',Float()),
                          **jobsettings)
et2._functions = [bif.PythonExpression('5*df["TestData"]','TestOut')]

# make sure the results of the python expression is saved to the derived metrics table
et2._data_items.append({'columnName': 'TestOut', 'columnType': 'NUMBER', 'kpiFunctionId': 22856, 
                         'kpiFunctionDto': {'output': {'name': 'TestOut'}},
                        'name': 'TestOut', 'parentDataItemName': None, 'sourceTableName': 'dm_markus_testdata',
                        'transient': False,'type': 'DERIVED_METRIC'})
# map device id to entity id for the derived metrics table
et2._data_items.append({'columnName': 'deviceid', 'columnType': 'LITERAL', 'kpiFunctionId': None,
                         'kpiFunctionDto': {},
                         'name': 'ENTITY_ID', 'parentDataItemName': None,'sourceTableName': 'dm_markus_testdata',
                         'transient': False,'type': 'METRIC'})

# make sure the results of the python expression is saved to the derived metrics daily table
et2._data_items.append({'columnName': 'TestData_max', 'columnType': 'NUMBER', 'kpiFunctionId': 22856, 
                         'kpiFunctionDto': {'output': {'name': 'TestData_max'}},
                        'name': 'TestData_max', 'parentDataItemName': None, 'sourceTableName': 'dm_markus_testdata_daily',
                        'transient': False,'type': 'DERIVED_METRIC'})
# map device id to entity id for the derived metrics daily table
et2._data_items.append({'columnName': 'deviceid', 'columnType': 'LITERAL', 'kpiFunctionId': None,
                         'kpiFunctionDto': {},
                         'name': 'ENTITY_ID', 'parentDataItemName': None,'sourceTableName': 'dm_markus_testdata_daily',
                         'transient': False,'type': 'METRIC'})


2019-09-27T14:01:16.541 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 2.0.3
2019-09-27T14:01:16.542 DEBUG iotfunctions.pipeline.__init__ Starting trace
2019-09-27T14:01:16.543 DEBUG iotfunctions.pipeline.__init__ Trace name: auto_trace_markus_testdata_20190927120116
2019-09-27T14:01:16.543 DEBUG iotfunctions.pipeline.__init__ auto_save None
2019-09-27T14:01:16.544 WARNING iotfunctions.metadata.__init__ No _db_schema specified in **kwargs. Usingdefault database schema.
2019-09-27T14:01:16.545 DEBUG iotfunctions.util.categorize_args categorizing arguments


/home/markus/.local/lib/python3.7/site-packages/sqlalchemy/engine/reflection.py:913: SAWarning: index key 'sqlnotapplicable' was not located in columns for table 'markus_testdata'
  "columns for table '%s'" % (flavor, c, table_name)


2019-09-27T14:01:19.783 DEBUG iotfunctions.metadata.__init__ Initialized entity type 
EntityType:markus_testdata
Functions:
Granularities:
No schedules metadata
2019-09-27T14:01:19.784 DEBUG iotfunctions.base.parse_expression expression (5*df["TestData"])


In [9]:
# dm_markus_testdate MUST exist, so run the following sql statment in DBeaver
#CREATE TABLE BLUADMIN.DM_MARKUS_TESTDATA (
#  entity_id varchar(255),
#  key varchar(255),
#  value_n double,
#  value_b boolean,
#  value_s varchar(255),
#  value_t timestamp,
#  timestamp timestamp,
#  last_update timestamp
# );


# The commented out version just dumps the job spec
#jobsettings = {'writer_name' : Db2DataWriter, 'db': db, '_db_schema': 'BLUADMIN', 'save_trace_to_file' : True, '_get_job_spec': True}
jobsettings = {'writer_name' : Db2DataWriter, 'db': db, '_db_schema': 'BLUADMIN', 'save_trace_to_file' : True}
job = pp.JobController(et2, **jobsettings)
job.execute()

2019-09-27T14:04:42.995 DEBUG iotfunctions.pipeline.set_payload_params Setting param writer_name on payload to <class 'iotfunctions.pipeline.Db2DataWriter'>
2019-09-27T14:04:42.996 DEBUG iotfunctions.pipeline.set_payload_params Setting param db on payload to <iotfunctions.db.Database object at 0x7f34287397f0>
2019-09-27T14:04:42.997 DEBUG iotfunctions.pipeline.set_payload_params Setting param _db_schema on payload to BLUADMIN
2019-09-27T14:04:42.998 DEBUG iotfunctions.pipeline.set_payload_params Setting param save_trace_to_file on payload to True
2019-09-27T14:04:42.999 DEBUG iotfunctions.pipeline.set_payload_params Setting param tenant_id on payload to AnalyticsServiceDev
2019-09-27T14:04:44.291 DEBUG iotfunctions.pipeline.get_output_list The payload has candidate data items ['deviceid', 'evt_timestamp', 'TestData', 'deviceid', 'deviceid']. The DataReader has no projection list
2019-09-27T14:04:44.292 DEBUG iotfunctions.metadata.classify_stages Input set was preset for function Python

/home/markus/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-09-27T14:04:47.691 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "PUT /analytics-runtime-analyticsservicedev-799d2008b460/AnalyticsServiceDev/markus_testdata/20190927/markus_testdata_trace_120444 HTTP/1.1" 200 0
2019-09-27T14:04:47.696 DEBUG iotfunctions.pipeline.save Saved trace to cos AnalyticsServiceDev/markus_testdata/20190927/markus_testdata_trace_120444
2019-09-27T14:04:47.698 DEBUG iotfunctions.pipeline.save wrote trace to file auto_trace_markus_testdata_20190927120444.json
2019-09-27T14:04:48.280 DEBUG iotfunctions.pipeline.update Updated job log (markus_testdata,5min): 2019-09-27 12:04:44.297393
2019-09-27T14:04:48.283 DEBUG iotfunctions.pipeline.get_next_future_execution Next scheduled execution date is 2019-09-27 12:09:44.297393
2019-09-27T14:04:48.284 DEBUG iotfunctions.pipeline.execute Ending job normally as there are no scheduled executions  due before execution end time
2019-09-27T14:04:48.380 DEBUG iotfunctions.pip

In [13]:
print (et2.get_data_items())

[{'name': 'deviceid', 'type': 'METRIC', 'parentDataItem': None, 'kpiFunctionDto': None, 'columnName': 'deviceid', 'columnType': 'LITERAL', 'sourceTableName': 'markus_testdata', 'tags': [], 'transient': False}, {'name': 'evt_timestamp', 'type': 'METRIC', 'parentDataItem': None, 'kpiFunctionDto': None, 'columnName': 'evt_timestamp', 'columnType': 'TIMESTAMP', 'sourceTableName': 'markus_testdata', 'tags': [], 'transient': False}, {'name': 'TestData', 'type': 'METRIC', 'parentDataItem': None, 'kpiFunctionDto': None, 'columnName': 'TestData', 'columnType': 'NUMBER', 'sourceTableName': 'markus_testdata', 'tags': [], 'transient': False}, {'columnName': 'TestOut', 'columnType': 'NUMBER', 'kpiFunctionId': 22856, 'kpiFunctionDto': {'output': {'name': 'TestOut'}}, 'name': 'TestOut', 'parentDataItemName': None, 'sourceTableName': 'dm_markus_testdata', 'transient': False, 'type': 'DERIVED_METRIC'}, {'columnName': 'deviceid', 'columnType': 'LITERAL', 'kpiFunctionId': None, 'kpiFunctionDto': {}, 'nam

In [7]:
# dm_markus_testdate MUST exist, so run the following sql statment in DBeaver
#CREATE TABLE BLUADMIN.DM_MARKUS_TESTDATA_DAILY (
#  entity_id varchar(255),
#  key varchar(255),
#  value_n double,
#  value_b boolean,
#  value_s varchar(255),
#  value_t timestamp,
#  timestamp timestamp,
#  last_update timestamp
# );

from iotfunctions.metadata import Granularity
from iotfunctions.pipeline import AggregateItems
daily = Granularity(
    name = 'daily',
    freq = '1D',                 # pandas frequency string
    timestamp= 'evt_timestamp',      # build time aggregations using this datetime col
    entity_id = 'deviceid',            # aggregate by id
    dimensions = None,
    entity_name = None
)

#myAgg = bif.AggregateWithExpression(['TestData'],'x.max()','TestMax')
myAgg = AggregateItems(['TestData'], 'max')
myAgg.granularity = daily

et2._functions = [myAgg]
et2.grains = [daily]
#et2._granularities_dict['daily'] = daily

jobsettings = {'writer_name' : Db2DataWriter, '_db_schema': 'BLUADMIN', 'save_trace_to_file' : True}
job = pp.JobController(et2, **jobsettings)
#job.data_writer = DataWriterFile
job.execute()

2019-09-27T12:50:01.158 DEBUG iotfunctions.pipeline.set_payload_params Setting param writer_name on payload to <class 'iotfunctions.pipeline.Db2DataWriter'>
2019-09-27T12:50:01.159 DEBUG iotfunctions.pipeline.set_payload_params Setting param _db_schema on payload to BLUADMIN
2019-09-27T12:50:01.160 DEBUG iotfunctions.pipeline.set_payload_params Setting param save_trace_to_file on payload to True
2019-09-27T12:50:01.160 DEBUG iotfunctions.pipeline.set_payload_params Setting param tenant_id on payload to AnalyticsServiceDev
2019-09-27T12:50:02.361 DEBUG iotfunctions.pipeline.get_output_list The payload has candidate data items ['deviceid', 'evt_timestamp', 'TestData', 'deviceid', 'deviceid']. The DataReader has no projection list
2019-09-27T12:50:02.363 DEBUG iotfunctions.metadata.classify_stages Output list set was preset for function AggregateItems
2019-09-27T12:50:02.364 DEBUG iotfunctions.metadata.classify_stages Function AggregateItems has no _metadata_params property. This property

2019-09-27T12:50:05.053 INFO iotfunctions.pipeline.update_last_entry Trace payload: {'produces_output_items': False, 'output_items': None, 'discard_prior_data': False, 'new_data_items_info': 'Function is configured not to produce any new data items  during execution', 'can_proceed': True, 'updated': datetime.datetime(2019, 9, 27, 10, 50, 5, 53355), 'cumulative_usage': 170}
2019-09-27T12:50:05.054 DEBUG iotfunctions.pipeline.write Executing stage markus_testdata_input_level.
2019-09-27T12:50:05.055 DEBUG iotfunctions.pipeline.execute Data items will be written to database for interval (None, 2019-09-27 10:50:02.367674)
2019-09-27T12:50:05.057 WARNING iotfunctions.pipeline._get_active_cols_properties No function definition defined for data item deviceid.
2019-09-27T12:50:05.058 INFO iotfunctions.pipeline._get_active_cols_properties The column devicetype in data frame does not correspond to a data item. Therefore it is not written to the database.
2019-09-27T12:50:05.059 INFO iotfunctions

/home/markus/.local/lib/python3.7/site-packages/ibm_db_sa/reflection.py:255: SAWarning: Did not recognize type 'BOOLEAN' of column 'VALUE_B'
  (coltype, r[0]))
/home/markus/.local/lib/python3.7/site-packages/sqlalchemy/engine/reflection.py:913: SAWarning: index key 'sqlnotapplicable' was not located in columns for table 'dm_markus_testdata_daily'
  "columns for table '%s'" % (flavor, c, table_name)


2019-09-27T12:50:08.159 DEBUG iotfunctions.pipeline._get_table_properties For table dm_markus_testdata_daily: delete statement: DELETE FROM dm_markus_testdata_daily, insert statement: INSERT INTO dm_markus_testdata_daily (entity_id, "KEY", value_n, value_b, value_s, value_t, "TIMESTAMP", last_update) VALUES (:entity_id, :KEY, :value_n, :value_b, :value_s, :value_t, :TIMESTAMP, :last_update)
2019-09-27T12:50:08.160 DEBUG iotfunctions.pipeline._get_table_properties For table dm_markus_testdata_daily: Mapping between column name and dataframe index position: ('entity_id', 0), ('TIMESTAMP', 1)
2019-09-27T12:50:08.161 INFO iotfunctions.pipeline.execute The data items will be written into the following tables: dm_markus_testdata_daily
2019-09-27T12:50:08.161 DEBUG iotfunctions.pipeline._delete_old_data Deleting old data items from table dm_markus_testdata_daily for time range [None, 2019-09-27 10:50:02.367674]
2019-09-27T12:50:08.958 INFO iotfunctions.pipeline._delete_old_data 0 data items h

/home/markus/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-09-27T12:50:10.943 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "PUT /analytics-runtime-analyticsservicedev-799d2008b460/AnalyticsServiceDev/markus_testdata/20190927/markus_testdata_trace_105002 HTTP/1.1" 200 0
2019-09-27T12:50:10.946 DEBUG iotfunctions.pipeline.save Saved trace to cos AnalyticsServiceDev/markus_testdata/20190927/markus_testdata_trace_105002
2019-09-27T12:50:10.950 DEBUG iotfunctions.pipeline.save wrote trace to file auto_trace_markus_testdata_20190927105002.json
2019-09-27T12:50:11.518 DEBUG iotfunctions.pipeline.update Updated job log (markus_testdata,5min): 2019-09-27 10:50:02.367674
2019-09-27T12:50:11.520 DEBUG iotfunctions.pipeline.get_next_future_execution Next scheduled execution date is 2019-09-27 10:55:02.367674
2019-09-27T12:50:11.522 DEBUG iotfunctions.pipeline.execute Ending job normally as there are no scheduled executions  due before execution end time
2019-09-27T12:50:11.618 DEBUG iotfunctions.pip

In [10]:
print (et2.get_data())

2019-09-27T14:04:54.880 DEBUG iotfunctions.metadata.index_df Indexed dataframe on id, evt_timestamp
                                 deviceid       devicetype  \
id    evt_timestamp                                          
73004 2019-09-19 11:36:35.244129    73004  markus_testdata   
73002 2019-09-19 11:37:35.244129    73002  markus_testdata   
73001 2019-09-19 11:38:35.244129    73001  markus_testdata   
73004 2019-09-19 11:39:35.244129    73004  markus_testdata   
73000 2019-09-19 11:40:35.244129    73000  markus_testdata   
73003 2019-09-19 11:41:35.244129    73003  markus_testdata   
73004 2019-09-19 11:37:03.228606    73004  markus_testdata   
      2019-09-19 11:42:03.228606    73004  markus_testdata   
73003 2019-09-25 09:52:17.892082    73003  markus_testdata   
      2019-09-25 09:53:17.892082    73003  markus_testdata   
73001 2019-09-25 09:54:17.892082    73001  markus_testdata   
73004 2019-09-25 09:55:17.892082    73004  markus_testdata   
73002 2019-09-25 09:56:17.892082